Importamos las librerías Pandas y Numpy

# Chipotle

Nos han contactado para hacer un análisis de la cadena de restaurantes Chipotle y entender mejor posibles puntos de mejora... Por ahora solo nos pasan una muestra de los datos [aquí](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv). ¿Somos capaces de explorarlos para ver de qué se trata? Qué tipo de información hay, está completa, podemos pedirles que nos la manden de otro modo, que nos informen de algún campo más, etc...


In [1]:
%pip install pandas numpy

import pandas as pd
import numpy as np


Note: you may need to restart the kernel to use updated packages.


### Step 2. Tomaremos el fichero en [esta URL](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv) y lo leeremos como un dataframe.

Pista: [read_csv](https://numpy.org/doc/stable/user/absolute_beginners.html#importing-and-exporting-a-csv)

In [2]:
url = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"

df = pd.read_csv(url, sep='\t')

print(df.head())


   order_id  quantity                              item_name  \
0         1         1           Chips and Fresh Tomato Salsa   
1         1         1                                   Izze   
2         1         1                       Nantucket Nectar   
3         1         1  Chips and Tomatillo-Green Chili Salsa   
4         2         2                           Chicken Bowl   

                                  choice_description item_price  
0                                                NaN     $2.39   
1                                       [Clementine]     $3.39   
2                                            [Apple]     $3.39   
3                                                NaN     $2.39   
4  [Tomatillo-Red Chili Salsa (Hot), [Black Beans...    $16.98   


### Step 3. Veamos los tipos de datos. ¿Podríamos obtener el producto de mayor precio?

In [3]:
print(df.dtypes)

highest_priced_product = df.loc[df['item_price'].idxmax()]

print("\nProduct with the highest price:")
print(highest_priced_product)


order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

Product with the highest price:
order_id                                        250
quantity                                          1
item_name                          Steak Salad Bowl
choice_description    [Fresh Tomato Salsa, Lettuce]
item_price                                   $9.39 
Name: 607, dtype: object


### Step 4. ¿Qué productos cuestan más de $10?

### Step 4.1: ¿Y cuántos pedidos se han hecho con un producto de más de 10$? ¿Es lo mismo?

### Step 4.2: ¿Y cuántos pedidos se han hecho de más de 10$? ¿Es lo mismo?

### Step 4.3: ¿Y en cuántos pedidos se ha pagado más de 10$ por un mismo producto? ¿Es lo mismo?

### Step 5. ¿Qué precio tiene cada producto en distintos pedidos? ¿Hay productos con varios precios?

### Step 6. Ordena el dataframe en base al nombre de producto (item name)

### Step 7. ¿Cuantas veces se ha pedido los productos más caros?

### Step 8. Veamos para el caso de Veggie Salad Bowl. Extrae esa información.